Fligh Delay Predtiction using linear regression . 
In this model , we will see how to predict flight delays on various data . This tutorial will perform:
1. Data Cleansing
2. Data Manipulation and EDA
3. Chi Square test to determine realtionship between categorical variables
4. Correlation , Covariance of the data
5. Orinary Linear regresssion model using skikitlear 
6. Determine KPPI of model : Standard Error 
7. Run gradient dscent on the model

Mount the data , we have below files: 
1. Flights: Yea wise data consiting of categorical and continuous data
2. Airline : consiting ariline codes not much use in model
3. Airports : Airports data consisting langitude and latitiue

In [ ]:
from google.colab import drive
gdrive = drive.mount('/content/drive/')
import os
os.listdir()

Mounted at /content/drive/


['.config', 'drive', 'sample_data']

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Colab").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

As spark is faster way to read the data , however spark dataframe do not provide the ease and capability of pandas dataframe to plot graph , and perform EDA. 

Koalas from databrick bridge the gap betwwen Pandas and spark. INstall Koala from databricks as below

In [ ]:
! pip install koalas

     |████████████████████████████████| 390 kB 7.2 MB/s 


In [ ]:
import pyspark.pandas as ps

In [ ]:
flight_headers = ["YEAR", "MONTH", "DAY", "DAY_OF_WEEK", "AIRLINE", "FLIGHT_NUMBER", "TAIL_NUMBER", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "SCHEDULED_DEPARTURE", "DEPARTURE_TIME", "DEPARTURE_DELAY", "TAXI_OUT", "WHEELS_OFF", "SCHEDULED_TIME", "ELAPSED_TIME", "AIR_TIME", "DISTANCE", "WHEELS_ON", "TAXI_IN", "SCHEDULED_ARRIVAL", "ARRIVAL_TIME", "ARRIVAL_DELAY", "DIVERTED", "CANCELLED", "CANCELLATION_REASON", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY", "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]
flight_dtypes = {"YEAR" : "str" , "MONTH" : "str" , "DAY" : "str" , "DAY_OF_WEEK" : "str" , "AIRLINE" : "str" , "FLIGHT_NUMBER" : "str" , "TAIL_NUMBER" : "str" , "ORIGIN_AIRPORT" : "str" , "DESTINATION_AIRPORT" : "str" , "SCHEDULED_DEPARTURE" : "str" , "DEPARTURE_TIME" : "str" , "DEPARTURE_DELAY" : "str" , "TAXI_OUT" : "str" , "WHEELS_OFF" : "str" , "SCHEDULED_TIME" : "str" , "ELAPSED_TIME" : "str" , "AIR_TIME" : "str" , "DISTANCE" : "str" , "WHEELS_ON" : "str" , "TAXI_IN" : "str" , "SCHEDULED_ARRIVAL" : "str" , "ARRIVAL_TIME" : "str" , "ARRIVAL_DELAY" : "str" , "DIVERTED" : "str" , "CANCELLED" : "str" , "CANCELLATION_REASON" : "str" , "AIR_SYSTEM_DELAY" : "str" , "SECURITY_DELAY" : "str" , "AIRLINE_DELAY" : "str" , "LATE_AIRCRAFT_DELAY" : "str" , "WEATHER_DELAY" : "str"}

In [ ]:
flights = ps.read_csv("drive/MyDrive/DataScienseStudyNotes/flights.csv")#,header=None, names=flight_headers, dtype=flight_dtypes) #, header =[1])\n
airlines = ps.read_csv("drive/MyDrive/DataScienseStudyNotes/airlines.csv")#, header =[1])
airports = ps.read_csv("drive/MyDrive/DataScienseStudyNotes/airports.csv")#, header =[1]

EDA:  Idenitfy the datasets to get more insights. 
1) Check dimension of data : Number if tows, nulls, datatypes

Below function will give the columns with number of counts of nulls and their percentage with total data. Higher number of nulls will indicate low quality of the feature and should be avoided.

In [ ]:
def get_nulls_percentage (df):
  counts = df.isna().sum()
  counts = counts.sort_values()
  percentage = round(df.isna().mean() * 100, 1)
  percentages = percentage.sort_values()
  null_values = ps.concat([counts, percentages], axis=1, keys=["count", "%"])
  return null_values


Cobine all the datasets together and remove the redudant columns.

In [ ]:
flights_features = ps.merge(flights,airlines, left_on= 'AIRLINE', right_on='IATA_CODE')
flights_features = ps.merge(flights_features, airports, left_on= 'ORIGIN_AIRPORT', right_on = 'IATA_CODE')
flights_features = ps.merge(flights_features, airports, left_on= 'DESTINATION_AIRPORT', right_on = 'IATA_CODE')

Drop the unneccesary redudant  coloumns make uniform dataset

In [ ]:
flights_data = flights_features.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE_x', 'IATA_CODE_x', 'IATA_CODE_y' ], axis = 1)
flights_data.rename(columns={'AIRLINE_y': 'AIRLINE', 'AIRPORT_x': 'ORG_AIRPORT', 'CITY_x':  'ORG_CITY', 'STATE_x': 'ORG_STATE',  'COUNTRY_x': 'ORG_COUNTRY',  'LATITUDE_x': 'ORG_AIRPORT_LATITUDE', 'LONGITUDE_x': 'ORG_AIRPORT_LONGITUDE', 'AIRPORT_y' : 'DEST_AIRPORT', 'CITY_y': 'DEST_CITY', 'STATE_y':'DEST_STATE',	'COUNTRY_y':'DEST_COUNTRY', 'LATITUDE_y': 'DEST_LATITUDE', 'LONGITUDE_y': 'DEST_LONGITUDE'}, inplace=True)
#flights_data.head(20)

In [ ]:
American_Al = flights_data.loc[flights_data["AIRLINE"]== 'American Airlines Inc.']
American_Al.head(20)

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE,ORG_AIRPORT,ORG_CITY,ORG_STATE,ORG_COUNTRY,ORG_AIRPORT_LATITUDE,ORG_AIRPORT_LONGITUDE,IATA_CODE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
1,2015,1,1,4,2336,N3KUAA,10,2.0,-8.0,12.0,14.0,280,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559
3,2015,1,1,4,258,N3HYAA,20,15.0,-5.0,15.0,30.0,285,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
8,2015,1,1,4,1112,N3LAAA,30,19.0,-11.0,17.0,36.0,195,193.0,173.0,1464,529.0,3.0,545,532.0,-13.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
11,2015,1,1,4,1674,N853AA,35,27.0,-8.0,21.0,48.0,268,266.0,238.0,2174,746.0,7.0,803,753.0,-10.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
21,2015,1,1,4,371,N3GXAA,100,52.0,-8.0,30.0,122.0,338,347.0,311.0,2724,933.0,6.0,938,939.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
23,2015,1,1,4,115,N3CTAA,105,103.0,-2.0,14.0,117.0,286,276.0,255.0,2342,832.0,7.0,851,839.0,-12.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
29,2015,1,1,4,2392,N3HRAA,120,141.0,21.0,12.0,153.0,227,208.0,188.0,1709,701.0,8.0,707,709.0,2.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.66700,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
31,2015,1,1,4,2211,N3CGAA,127,116.0,-11.0,10.0,126.0,239,234.0,217.0,1972,703.0,7.0,726,710.0,-16.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Phoenix Sky Harbor International Airport,Phoenix,AZ,USA,33.43417,-112.00806,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
42,2015,1,1,4,2459,N3BDAA,200,NaN,NaN,NaN,NaN,120,NaN,NaN,868,NaN,NaN,500,NaN,NaN,0,1,B,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Phoenix Sky Harbor International Airport,Phoenix,AZ,USA,33.43417,-112.00806,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
64,2015,1,1,4,1323,N3CXAA,510,507.0,-3.0,14.0,521.0,63,55.0,38.0,192,559.0,3.0,613,602.0,-11.0,0,0,None,NaN,NaN,NaN,NaN,NaN,American Airlines Inc.,Orlando International Airport,Orlando,FL,USA,28.42889,-81.31603,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056


Get number of nulls and percentage of nulls. The features  having higher percentage of nulls will be outliers making the model overfitting. Below coloumns are having higher percentage of nulls , we will not consider them for further processings.

In [ ]:
#type(flights_data)
print(get_nulls_percentage(American_Al))

TypeError: ignored

Drop coloumns with higher number of nulls as that will lead overfitting / underfitting of model. features having higher number of nulls should be dropped.

In [ ]:
American_Al = American_Al.drop(['WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY','AIRLINE_DELAY', 'SECURITY_DELAY', 'AIR_SYSTEM_DELAY', 'CANCELLATION_REASON'], axis = 1)


Combine the airport code and their geolocations.

In [ ]:
print(get_nulls_percentage(American_Al))

In [ ]:
American_Al = American_Al.dropna()
#print(get_nulls_percentage(flight_features))

In [ ]:
def insertChar(mystring ):
     #mystring = mystring.split(".")[0]
     #mystring =  mystring[mystring.index('.'):]
     lenofstring = len(mystring)
     #print(\"len\" + str(len(mystring)))\n",
     if (lenofstring == 1):
            #print(\"1\")\n",
      mystring = '0' + mystring[0:] +':' + '00'
     elif (lenofstring == 2):
       if (int(mystring[0:2]) <= 24):
        mystring = mystring[:] + ':00'
       else :
        mystring =  '00:' +mystring 
     elif (lenofstring == 3):
       if (int(mystring[0:2]) <= 24):
        mystring = mystring[0:2] + ':' + mystring[2:3] + '0'
       else:    
        mystring = '0' + mystring[1:] + ':00'
     elif (lenofstring > 4):
       mystring = mystring[0:4]
       mystring = mystring[0:2] + ':' + mystring[2:] 
     elif (lenofstring == 4):
       mystring = mystring[0:2] + ':' + mystring[2:]
     return mystring 

In [ ]:
print(insertChar("5"))

05:00


Convert date day and time to single coumn. Convert Departure time and arrival time

In [ ]:
American_Al["TIMESTAMP"] = ps.to_datetime(American_Al['YEAR']+ American_Al['MONTH']+ American_Al['DAY'])
#flight_features["TIMESTAMP"] =flight_features[['YEAR', 'MONTH', 'DAY']]




In [ ]:
American_Al["DEPARTURE_TIME"] = American_Al["DEPARTURE_TIME"].astype(str).apply(insertChar)
American_Al["SCHEDULED_DEPARTURE"] =American_Al["SCHEDULED_DEPARTURE"].astype(str).apply(insertChar)
American_Al["SCHEDULED_ARRIVAL"] =American_Al["SCHEDULED_ARRIVAL"].astype(str).apply(insertChar)
American_Al["ARRIVAL_TIME"] =American_Al["ARRIVAL_TIME"].astype(str).apply(insertChar)


In [ ]:
American_Al.head(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,TAIL_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIRLINE,ORG_AIRPORT,ORG_CITY,ORG_STATE,ORG_COUNTRY,ORG_AIRPORT_LATITUDE,ORG_AIRPORT_LONGITUDE,IATA_CODE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE,TIMESTAMP
1,2015,1,1,4,2336,N3KUAA,10:00,02:00,-8,12,14,280,279,263,2330,737,4,050:00,041:00,-9,0,0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559,1970-01-01 00:00:00.000002
3,2015,1,1,4,258,N3HYAA,20:00,15:00,-5,15,30,285,281,258,2342,748,8,005:00,056:00,-9,0,0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
8,2015,1,1,4,1112,N3LAAA,00:30,19:00,-11,17,36,195,193,173,1464,529,3,045:00,032:00,-13,0,0,American Airlines Inc.,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,DFW,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,1970-01-01 00:00:00.000002
11,2015,1,1,4,1674,N853AA,00:35,00:27,-8,21,48,268,266,238,2174,746,7,003:00,053:00,-10,0,0,American Airlines Inc.,McCarran International Airport,Las Vegas,NV,USA,36.08036,-115.15233,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
21,2015,1,1,4,371,N3GXAA,10:00,00:52,-8,30,122,338,347,311,2724,933,6,038:00,039:00,1,0,0,American Airlines Inc.,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
23,2015,1,1,4,115,N3CTAA,10:50,10:30,-2,14,117,286,276,255,2342,832,7,051:00,039:00,-12,0,0,American Airlines Inc.,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
29,2015,1,1,4,2392,N3HRAA,12:00,14:10,21,12,153,227,208,188,1709,701,8,007:00,009:00,2,0,0,American Airlines Inc.,Denver International Airport,Denver,CO,USA,39.85841,-104.66700,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
31,2015,1,1,4,2211,N3CGAA,12:70,11:60,-11,10,126,239,234,217,1972,703,7,026:00,010:00,-16,0,0,American Airlines Inc.,Phoenix Sky Harbor International Airport,Phoenix,AZ,USA,33.43417,-112.00806,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
64,2015,1,1,4,1323,N3CXAA,010:00,007:00,-3,14,521,63,55,38,192,559,3,013:00,002:00,-11,0,0,American Airlines Inc.,Orlando International Airport,Orlando,FL,USA,28.42889,-81.31603,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002
70,2015,1,1,4,1057,N3ASAA,015:00,003:00,108,15,718,161,155,133,1121,1031,7,056:00,10:38,102,0,0,American Airlines Inc.,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,1970-01-01 00:00:00.000002


In [ ]:
American_Al.to_excel("drive/MyDrive/DataScienseStudyNotes/flightdata_cleansed.xlsx")

NameError: ignored